In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 200) # Display all columns

In [3]:
df = pd.read_parquet("../data/processed/churn_prediction_dataset.parquet")
df.head()

,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_anual,mcomisiones,mactivos_margen,mpasivos_margen,cant_total_prod,tpaquete1,tpaquete2,tpaquete3,tpaquete4,tpaquete5,tpaquete6,tpaquete7,tpaquete8,tpaquete9,mdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cfondos_comunes_inversion,mfondos_comunes_inversion_pesos,mfondos_comunes_inversion_dolares,ctitulos,mtitulos,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,mbonos_corporativos,mmonedas_extranjeras,minversiones_otras,cplan_sueldo,mplan_sueldo,mplan_sueldo_manual,cplan_sueldo_transaccion,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,ccambio_monedas,ccambio_monedas_compra,mcambio_monedas_compra,ccambio_monedas_venta,mcambio_monedas_venta,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,thomebanking,chomebanking_transacciones,cautoservicio,cautoservicio_transacciones,tmovimientos_ultimos90dias,visa_marca_atraso,visa_mfinanciacion_limite,visa_msaldototal,visa_msaldopesos,visa_msaldodolares,visa_mconsumospesos,visa_mconsumosdolares,visa_mlimitecompra,visa_mpagado,visa_mpagospesos,visa_mpagosdolares,visa_mconsumototal,visa_cconsumos,visa_mpagominimo,clase_binaria,visa_tenure_days,tcuentas_2,visa_cuenta_estado_11.0,visa_cuenta_estado_12.0,visa_cuenta_estado_19.0
0,0,1,44,72,3296.69,14455.27,579.66,910.65,1411.60,11,1,0,0,0,0,0,0,0,0,1.17,-460.04,2,2574.00,1,17,19451.75,1,18,17091.34,3,73704.79,0,0.0,0.0,0,0.0,0.0,0,0.00,0,1,0,0,0,0,0,1,56580.03,0.0,0,1,68.94,0,0.0,0,0.00,0,0.0,0.00,14,579.66,0,0,0.0,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,116,0.0,63180.0,29994.31,32262.86,2830.47,14555.47,59.47,70200.0,0.0,-19632.60,0.00,14555.47,13.0,3837.6,0,4203,False,False,False,False
1,0,0,78,159,1032.18,10141.18,790.58,71.21,135.29,6,1,0,0,0,0,0,0,0,0,1.17,6301.72,0,0.00,1,1,174.33,1,0,0.00,0,0.00,0,0.0,0.0,0,0.0,0.0,0,0.00,0,0,0,0,0,0,0,0,0.00,0.0,0,0,0.00,0,0.0,0,0.00,0,0.0,833.97,4,790.58,0,0,0.0,0,0.0,0,0.0,1,351.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,1,0,0,17,0.0,63180.0,182.45,213.46,0.00,-11.55,4.22,70200.0,0.0,-2976.51,62.78,-11.55,2.0,35.1,0,6859,False,False,False,False
2,0,1,64,299,-52.34,4752.64,236.08,-422.18,175.66,7,1,0,0,0,0,0,0,0,0,1.17,45.63,0,0.00,1,8,11670.93,1,0,0.00,0,0.00,0,0.0,0.0,0,0.0,0.0,0,0.00,0,0,0,0,0,0,0,0,0.00,0.0,0,0,0.00,0,0.0,0,0.00,0,0.0,0.00,6,236.08,0,0,0.0,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,0,0,0,10,0.0,176904.0,10367.71,12130.22,0.00,6788.67,0.00,196560.0,0.0,-12753.00,0.00,6788.67,4.0,1977.3,0,10314,False,False,False,False
3,0,1,48,282,485.91,10676.87,1280.47,-1275.18,596.07,8,1,0,0,0,0,0,0,0,0,1.17,5429.85,4,4103.69,1,9,9840.21,1,14,12383.23,0,0.00,0,0.0,0.0,0,0.0,0.0,3,10509.33,0,0,0,0,0,0,0,0,0.00,0.0,0,4,5303.31,0,0.0,0,0.00,2,2460.0,833.97,18,1280.47,0,0,0.0,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,78,0,0,85,0.0,176904.0,6754.10,7433.72,468.57,2946.70,10.51,196560.0,0.0,-3685.76,0.00,2946.70,5.0,573.3,0,3578,False,False,False,False
4,0,1,45,171,5878.27,24298.75,64.38,22.11,4947.03,9,1,0,0,0,0,0,0,0,0,1.17,563006.87,10,4313.45,1,2,1496.03,1,0,0.00,0,0.00,1,351000.0,0.0,0,0.0,0.0,0,0.00,0,0,0,0,0,0,0,0,0.00,0.0,0,0,0.00,0,0.0,7,2171.89,1,339.3,0.00,12,64.38,0,0,0.0,0,0.0,0,0.0,2,16160.54,5,4797.0

In [4]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score

# Features and target
X = df.drop(columns=["clase_binaria"])  # Exclude the target column
y = df["clase_binaria"]

# Define the XGBoost model with class balancing
xgb_model = xgb.XGBClassifier(scale_pos_weight=1, random_state=12, n_estimators=100)

# Define StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

# Perform stratified cross-validation with AUC scoring
cv_auc_scores = cross_val_score(xgb_model, X, y, cv=skf, scoring='roc_auc')  # Using AUC as the metric

# Perform stratified cross-validation with Accuracy scoring
cv_accuracy_scores = cross_val_score(xgb_model, X, y, cv=skf, scoring='accuracy')  # Using Accuracy as the metric

# Print Cross-Validation AUC Scores
print("Cross-Validation AUC Scores:", cv_auc_scores)

# Print Cross-Validation Accuracy Scores
print("Cross-Validation Accuracy Scores:", cv_accuracy_scores)

# Calculate and print the Mean AUC Score
print("Mean AUC Score:", cv_auc_scores.mean())

# Calculate and print the Mean Accuracy Score
print("Mean Accuracy Score:", cv_accuracy_scores.mean())

# Calculate and print the validation errors (1 - AUC score)
cv_auc_errors = 1 - cv_auc_scores
print("Validation Errors (AUC):", cv_auc_errors)
print("Mean Validation Error (AUC):", cv_auc_errors.mean())

# Calculate and print the validation errors (1 - accuracy score)
cv_accuracy_errors = 1 - cv_accuracy_scores
print("Validation Errors (Accuracy):", cv_accuracy_errors)
print("Mean Validation Error (Accuracy):", cv_accuracy_errors.mean())

# Fit the XGBoost model on the entire dataset (optional, for evaluation purposes)
xgb_model.fit(X, y)
y_pred = xgb_model.predict(X)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=["No Churn", "Churn"]))


Cross-Validation AUC Scores: [0.93728772 0.94780681 0.92716024 0.93602409 0.92976706]
Cross-Validation Accuracy Scores: [0.99362187 0.99432025 0.9940469  0.99422913 0.99407727]
Mean AUC Score: 0.9356091861910848
Mean Accuracy Score: 0.9940590838180787
Validation Errors (AUC): [0.06271228 0.05219319 0.07283976 0.06397591 0.07023294]
Mean Validation Error (AUC): 0.06439081380891525
Validation Errors (Accuracy): [0.00637813 0.00567975 0.0059531  0.00577087 0.00592273]
Mean Validation Error (Accuracy): 0.005940916181921274

Classification Report:
              precision    recall  f1-score   support

    No Churn       1.00      1.00      1.00    163292
       Churn       0.99      0.65      0.79      1329

    accuracy                           1.00    164621
   macro avg       1.00      0.83      0.89    164621
weighted avg       1.00      1.00      1.00    164621



In [21]:
X_test

,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_anual,mcomisiones,mactivos_margen,mpasivos_margen,cant_total_prod,tpaquete1,tpaquete2,tpaquete3,tpaquete4,tpaquete5,tpaquete6,tpaquete7,tpaquete8,tpaquete9,mdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cfondos_comunes_inversion,mfondos_comunes_inversion_pesos,mfondos_comunes_inversion_dolares,ctitulos,mtitulos,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,mbonos_corporativos,mmonedas_extranjeras,minversiones_otras,cplan_sueldo,mplan_sueldo,mplan_sueldo_manual,cplan_sueldo_transaccion,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,ccambio_monedas,ccambio_monedas_compra,mcambio_monedas_compra,ccambio_monedas_venta,mcambio_monedas_venta,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,thomebanking,chomebanking_transacciones,cautoservicio,cautoservicio_transacciones,tmovimientos_ultimos90dias,visa_marca_atraso,visa_mfinanciacion_limite,visa_msaldototal,visa_msaldopesos,visa_msaldodolares,visa_mconsumospesos,visa_mconsumosdolares,visa_mlimitecompra,visa_mpagado,visa_mpagospesos,visa_mpagosdolares,visa_mconsumototal,visa_cconsumos,visa_mpagominimo,visa_tenure_days,tcuentas_2,visa_cuenta_estado_11.0,visa_cuenta_estado_12.0,visa_cuenta_estado_19.0
47532,0,0,57,209,1420.39,14893.02,178.00,752.43,309.44,11,1,0,0,0,0,0,0,0,0,1.17,-4129.18,0,0.00,1,9,8174.26,1,0,0.00,3,127072.99,0,0.0,0.0,0,0.00,0.0,0,0.0,0,0,1,0,0,0,0,2,27978.21,0.0,0,1,22.23,0,0.0,0,0.0,0,0.00,0.00,11,178.00,0,0,0.0,0,0.0,0,0.0,0,0.00,12,18837.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,84,0.0,63180.00,7357.96,8608.81,0.00,4193.12,6.04,70200.00,0.0,-6647.68,0.00,4193.12,6.0,854.1,6540,0,0,0,0
64483,0,1,60,179,618.53,8029.68,273.59,-831.83,1126.65,10,1,0,0,0,0,0,0,0,0,1.17,24145.96,4,8877.59,1,15,14314.31,1,7,9266.59,1,4368.60,0,0.0,0.0,0,0.00,0.0,0,0.0,0,0,0,0,0,0,0,1,32931.99,0.0,0,2,91.18,0,0.0,0,0.0,0,0.00,0.00,10,273.59,0,0,0.0,0,0.0,0,0.0,0,0.00,2,3510.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,77,0.0,88452.00,12280.45,14368.12,0.00,4117.66,6.24,98280.00,0.0,-13967.41,0.00,4117.66,6.0,1076.4,6317,0,0,0,0
41972,0,1,58,141,1023.99,99883.12,281.93,-708.40,1342.64,8,1,0,0,0,0,0,0,0,0,1.17,251176.89,10,2158.59,1,40,26146.51,1,0,0.00,0,0.00,0,0.0,0.0,0,0.00,0.0,0,0.0,0,0,0,1,0,0,0,0,0.00,0.0,0,8,12676.79,0,0.0,0,0.0,4,6551.66,0.00,6,281.93,1,1,234.0,0,0.0,0,0.0,1,2106.00,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,1,5,0,0,180,0.0,415588.56,22368.24,26170.84,0.00,25571.28,2.67,461765.07,0.0,-13827.80,0.00,25571.28,40.0,1275.3,5117,0,0,0,0
116588,0,1,27,118,324.60,3993.23,77.51,-59.63,270.82,7,1,0,0,0,0,0,0,0,0,1.17,22969.33,4,1922.91,1,11,6498.25,1,0,0.00,0,0.00,0,0.0,0.0,0,0.00,0.0,0,0.0,0,0,0,0,0,0,0,1,41940.99,0.0,0,1,22.23,0,0.0,0,0.0,5,6258.42,0.00,10,77.51,0,0,0.0,0,0.0,0,0.0,3,13349.70,6,17550.0,0,0.0,0,0.0,0,0.0,0,0.0,1,37,0,0,147,0.0,50544.00,5679.64,6645.17,0.00,2718.09,16.98,56160.00,0.0,-4345.83,0.00,2718.09,10.0,397.8,5413,0,0,0,0
75284,0,1,35,168,4076.93,17348.30,609.67,1209.81,1753.66,6,1,0,0,0,0,0,0,0,0,1.17,330116.57,8,15764.58,1,3,1407.51,1,2,14207.97,0,0.00,0,0.0,0.0,0,0.00,0.0,0,0.0,0,0,0,0,0,0,0,1,75631.14,0.0,0,3,5470.94,0,0.0,0,0.0,2,3567.02,0.00,12,609.67,0,0,0.0,0,0.0,0,0.0,2,18098.58,0,0.0,0,0.0,0,0.0,0,0.0,0

In [5]:
# Boolean columns (bool) need to be converted to integers for SHAP to process them:
X = X.astype({col: 'int64' for col in X.select_dtypes(include=['bool']).columns})


In [7]:
# Add SHAP graphs

In [6]:
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
conf_matrix = confusion_matrix(y, y_pred)

# Print the confusion matrix with labels
print("Confusion Matrix:")
print(f"                Predicted No   Predicted Yes")
print(f"Actual No     TN = {conf_matrix[0, 0]}    FP = {conf_matrix[0, 1]}")
print(f"Actual Yes    FN = {conf_matrix[1, 0]}    TP = {conf_matrix[1, 1]}")


Confusion Matrix:
                Predicted No   Predicted Yes
Actual No     TN = 163286    FP = 6
Actual Yes    FN = 463    TP = 866


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, stratify=y)

# Train the model on the training set
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # For ROC AUC

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["No Churn", "Churn"]))

# Calculate and print ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC Score: {roc_auc:.4f}")


Confusion Matrix:
[[32635    24]
 [  160   106]]

Classification Report:
              precision    recall  f1-score   support

    No Churn       1.00      1.00      1.00     32659
       Churn       0.82      0.40      0.54       266

    accuracy                           0.99     32925
   macro avg       0.91      0.70      0.77     32925
weighted avg       0.99      0.99      0.99     32925


ROC AUC Score: 0.9432


In [17]:
# Define cost-benefit values
benefit_tp = 100   # Benefit of correctly predicting churn
cost_fn = -500     # Cost of missing a churner
cost_fp = -20      # Cost of wrongly targeting a non-churner
benefit_tn = 0     # No benefit or cost for correctly predicting no churn

In [18]:
import numpy as np
from sklearn.metrics import roc_curve

# Predict probabilities for the test set
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate the net benefit for different thresholds
thresholds = np.linspace(0, 1, 101)  # 101 thresholds from 0 to 1
net_benefits = []

for threshold in thresholds:
    # Apply the threshold
    y_pred_adjusted = (y_pred_proba >= threshold).astype(int)
    
    # Calculate confusion matrix components
    tp = ((y_pred_adjusted == 1) & (y_test == 1)).sum()
    fn = ((y_pred_adjusted == 0) & (y_test == 1)).sum()
    fp = ((y_pred_adjusted == 1) & (y_test == 0)).sum()
    tn = ((y_pred_adjusted == 0) & (y_test == 0)).sum()
    
    # Compute net benefit
    net_benefit = (benefit_tp * tp + benefit_tn * tn + cost_fp * fp + cost_fn * fn)
    net_benefits.append(net_benefit)

# Find the threshold with the maximum net benefit
optimal_threshold = thresholds[np.argmax(net_benefits)]
print(f"Optimal Threshold: {optimal_threshold:.2f}")


Optimal Threshold: 0.03


In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# Define cost-benefit values
benefit_tp = 100
cost_fn = -500
cost_fp = -20
benefit_tn = 0

# Create sample weights
sample_weights = np.where(y_train == 1, -cost_fn, -cost_fp)

# Train XGBoost model with sample weights
xgb_model = XGBClassifier(scale_pos_weight=1, random_state=12, n_estimators=100)
xgb_model.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluate the model
y_pred = xgb_model.predict(X_test)


In [26]:
# Adjust predictions using the optimal threshold
y_pred_final = (y_pred_proba >= optimal_threshold).astype(int)

# Evaluate the refined model
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final))


[[31603  1058]
 [  257     7]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     32661
           1       0.01      0.03      0.01       264

    accuracy                           0.96     32925
   macro avg       0.50      0.50      0.50     32925
weighted avg       0.98      0.96      0.97     32925



In [30]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import numpy as np

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# Define the objective function
def objective(trial):
    # Define the hyperparameter search space
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1, 10),
    }

    # Create the model
    model = XGBClassifier(**params, random_state=12, use_label_encoder=False, eval_metric="logloss")
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc")
    return np.mean(cv_scores)

# Create the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best hyperparameters:", study.best_params)

# Train the final model with the best parameters
best_params = study.best_params
final_model = XGBClassifier(**best_params, random_state=12, use_label_encoder=False, eval_metric="logloss")
final_model.fit(X_train, y_train)

# Evaluate the model
y_pred = final_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC AUC on test set:", roc_auc)


[I 2024-11-21 16:24:27,572] A new study created in memory with name: no-name-55a10a31-f8d3-4911-b5f9-89263e6df459
/Users/juanignaciomonge/Desktop/Repositories/uba-assessment-churn-prediction/venv_tp_final_ds/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:24:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/juanignaciomonge/Desktop/Repositories/uba-assessment-churn-prediction/venv_tp_final_ds/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:24:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/juanignaciomonge/Desktop/Repositories/uba-assessment-churn-prediction/venv_tp_final_ds/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:24:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "

Best hyperparameters: {'n_estimators': 112, 'max_depth': 5, 'learning_rate': 0.06360500546786668, 'subsample': 0.7925231934966257, 'colsample_bytree': 0.9516132188568334, 'gamma': 4.135689937932744, 'reg_alpha': 4.269469779030872, 'reg_lambda': 4.262171480466699, 'scale_pos_weight': 2.3952123164856607}


/Users/juanignaciomonge/Desktop/Repositories/uba-assessment-churn-prediction/venv_tp_final_ds/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:28:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


ROC AUC on test set: 0.9408077923912915


In [37]:
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1] 

In [42]:
y_pred_final = (y_pred_proba >= 0.5).astype(int)

In [44]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final))

[[32494   171]
 [   35   225]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     32665
           1       0.57      0.87      0.69       260

    accuracy                           0.99     32925
   macro avg       0.78      0.93      0.84     32925
weighted avg       1.00      0.99      0.99     32925

